# Colab initialization
- install the pipeline in the colab runtime
- download files neccessary for this example

In [1]:
!pip3 install -U pip > /dev/null
!pip3 install -U "bio-embeddings[all] @ git+https://gitlab.lrz.de/sacdallago/bio_embeddings.git@light_attention" > /dev/null

The system cannot find the path specified.
The system cannot find the path specified.


In [8]:
!wget http://data.bioembeddings.com/public/embeddings/feature_models/light_attention/la_bert_subcellular_localization.pt --output-document la_bert_subcellular_localization.pt
!wget http://data.bioembeddings.com/public/embeddings/feature_models/light_attention/la_bert_solubility.pt --output-document la_bert_solubility.pt

--2021-04-04 07:51:13--  http://data.bioembeddings.com/public/embeddings/feature_models/light_attention/la_bert_subcellular_localization.pt
Resolving data.bioembeddings.com (data.bioembeddings.com)... 143.95.108.236
Connecting to data.bioembeddings.com (data.bioembeddings.com)|143.95.108.236|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227320161 (217M) [text/plain]
Saving to: ‘localization_unprocessed.pt’

localization_unproc 100%[===================>] 216.79M  76.3MB/s    in 2.8s    

2021-04-04 07:51:16 (76.3 MB/s) - ‘localization_unprocessed.pt’ saved [227320161/227320161]

--2021-04-04 07:51:16--  http://data.bioembeddings.com/public/embeddings/feature_models/light_attention/la_bert_solubility.pt
Resolving data.bioembeddings.com (data.bioembeddings.com)... 143.95.108.236
Connecting to data.bioembeddings.com (data.bioembeddings.com)|143.95.108.236|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227317089 (217M) [text/plain]
Saving

# Extract solubility and subcellular localization predictions from ProtTrans T5 using Light Attention
In this notebook we will extract annotations from ProtTrans T5 embeddings via trained Light Attention models that can predict solubility and subcellular localization

In [14]:
from bio_embeddings.embed import ProtTransBertBFDEmbedder

We initialize the ProtTrans T5 embedder.

In [15]:
embedder = ProtTransBertBFDEmbedder()

prottrans_bert_bfd.zip: 1.56GB [00:18, 84.5MB/s]                            


We select an AA sequence. In this case, the sequence is that of [Aspartate aminotransferase, mitochondrial](https://www.uniprot.org/uniprot/P12345)


In [16]:
target_sequence = "MALLHSARVLSGVASAFHPGLAAAASARASSWWAHVEMGPPDPILGVTEAYKRDTNSKKMNLGVGAYRDDNGKPYVLPSVRKAEAQIAAKGLDKEYLPIGGLAEFCRASAELALGENSEVVKSGRFVTVQTISGTGALRIGASFLQRFFKFSRDVFLPKPSWGNHTPIFRDAGMQLQSYRYYDPKTCGFDFTGALEDISKIPEQSVLLLHACAHNPTGVDPRPEQWKEIATVVKKRNLFAFFDMAYQGFASGDGDKDAWAVRHFIEQGINVCLCQSYAKNMGLYGERVGAFTVICKDADEAKRVESQLKILIRPMYSNPPIHGARIASTILTSPDLRKQWLQEVKGMADRIIGMRTQLVSNLKKEGSTHSWQHITDQIGMFCFTGLKPEQVERLTKEFSIYMTKDGRISVAGVTSGNVGYLAHAIHQVTK"

We produce the embeddings of the above sequence. Since we only have one sequence, we use the simple `embed` function, rather than the `embed_many` or `embed_batch`, which we would instead use if we had multiple sequences to embed.

In [17]:
embedding = embedder.embed(target_sequence)

The `bio_embeddings` pipeline includes some models trained on embeddings for the prediction of Secondary Structure and Subcellular Localization. In the following we make use of these models.

To speed up processing, we have downloaded the model weights of the supervised subcellular localization and secondary structure prediction models from [here](http://maintenance.dallago.us/public/embeddings/feature_models/seqvec/).

In [19]:
from bio_embeddings.extract.light_attention import LightAttentionAnnotationExtractor

In [20]:
annotations_extractor = LightAttentionAnnotationExtractor(subcellular_location_checkpoint_file='la_bert_subcellular_localization.pt', membrane_checkpoint_file='la_bert_solubility.pt')

In [ ]:
annotations = annotations_extractor.get_subcellular_location(embedding)

Let's see what annotations are available from Light Attention

In [33]:
annotations._fields

('localization', 'membrane')

Let's print the subcellular localization predicted via the Light Attention on the ProtBert embeddings

In [36]:
print(f"The subcellular localization predicted from the embedding is: {annotations.localization.value}")
print(f"The solubility/membrane-boundness predicted from the embedding is: {annotations.membrane.value}")

The subcellular localization predicted from the embedding is: Mitochondrion
The embedding is soluble: Soluble
